In [188]:
# Загрузим нужные библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import plt, mpl
import seaborn as sns
mpl.rcParams['font.family'] = 'serif'
import math
%matplotlib inline
plt.style.use('fivethirtyeight')
# Чистка отклонений
import scipy.stats as stats
# Уберем warnings
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
# Тематическое моделирование
import gensim

from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
import random

In [219]:
# Тематическое моделирование
#test_df = them_topic(test_df, features='features')
#train_df = them_topic(train_df, features='features')



train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')




# Чистка широты и долготы
train_df["longitude"]=train_df["longitude"].apply(lambda x:-73.75 if x>=-73.75 else x)
test_df["longitude"]=test_df["longitude"].apply(lambda x:-74.05 if x<=-74.05 else x)
train_df["latitude"]=train_df["latitude"].apply(lambda x:40.4 if x<=40.4 else x)
test_df["latitude"]=test_df["latitude"].apply(lambda x:40.9 if x>=40.9 else x)

mean_price = int(train_df['price'].mean())
test_df.loc[test_df['price']<200,'price'] = mean_price
train_df.loc[train_df['price']<200,'price'] = mean_price
# Метапризнак предсказ цены на основе координат
#X_train_coor = train_df[['latitude', 'longitude']]
#X_test_coor = test_df[['latitude', 'longitude']]

#y_train_price = train_df.price
#y_test_price = test_df.price

#neigh_train = KNeighborsRegressor(n_neighbors=2)
#neigh_train.fit(X_train_coor, y_train_price)

#neigh_test = KNeighborsRegressor(n_neighbors=2)
#neigh_test.fit(X_test_coor, y_test_price)

#train_df['price_comparison'] = train_df['price'] /  neigh_train.predict(X_train_coor)
#test_df['price_comparison'] = test_df['price'] /  neigh_test.predict(X_test_coor)




train_df['features'] = [','.join(map(str, i)) for i in train_df['features']]
test_df['features'] = [','.join(map(str, i)) for i in test_df['features']]



# Функция расчета расстояния от манхэттэна
def distance_pair(lat1, lon1, lat2, lon2):
    # Перевод в радианы
    p = 0.017453292519943295     #Pi/180
    haver_formula = 0.5 - np.cos((lat2 - lat1) * p)/2 +  np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    d_2_point = 6367 *2 * np.arcsin(np.sqrt(haver_formula)) 
    return d_2_point
distance_pairs = np.vectorize(distance_pair)

# Расстояние от финансового центра
train_df['distance_manh'] = distance_pairs(train_df.latitude, train_df.longitude, 40.70858733058446, -74.01098113951349)
test_df['distance_manh'] = distance_pairs(test_df.latitude, test_df.longitude, 40.70858733058446, -74.01098113951349)


# Расстояние от центр парка
train_df['distance_central_park'] = distance_pairs(train_df.latitude, train_df.longitude, 40.77897128595648, -73.96656147253005)
test_df['distance_central_park'] = distance_pairs(test_df.latitude, test_df.longitude, 40.77897128595648, -73.96656147253005)

# Расстояние от  Бронкс
train_df['bronx'] = distance_pairs(train_df.latitude, train_df.longitude, 40.84928609864644, -73.88844362698887)
test_df['bronx'] = distance_pairs(test_df.latitude, test_df.longitude, 40.84928609864644, -73.88844362698887)

# Расстояние от Район Queens

train_df['queens'] = distance_pairs(train_df.latitude, train_df.longitude, 40.747803602867066, -73.90170458998642)
test_df['queens'] = distance_pairs(test_df.latitude, test_df.longitude, 40.747803602867066, -73.90170458998642)

# Расстояние от Район Brooklin кладбище

train_df['brooklin_cometery'] = distance_pairs(train_df.latitude, train_df.longitude, 40.68822862016868, -73.87615328604328)
test_df['brooklin_cometery'] = distance_pairs(test_df.latitude, test_df.longitude, 40.68822862016868, -73.87615328604328)

# Расстояние от Район Brooklin
train_df['brooklin'] = distance_pairs(train_df.latitude, train_df.longitude, 40.62506864788317, -73.96786745767038)
test_df['brooklin'] = distance_pairs(test_df.latitude, test_df.longitude, 40.62506864788317, -73.96786745767038)






# Кластеризация координат
geo = ['latitude', 'longitude']
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=42)

# Метки кластеров внесем в переменную clusters
train_df['clusters'] = kmeans.fit(train_df[geo]).labels_
test_df['clusters'] = kmeans.fit(test_df[geo]).labels_


#test_df = test.copy()


train_df['photos'] = train_df['photos'].apply(len)
test_df['photos'] = test_df['photos'].apply(len)

# Число признаков
train_df['num_features'] = train_df['features'].apply(len)
test_df['num_features'] = test_df['features'].apply(len)

# Количество слов
train_df['num_description_words'] = train_df['description'].apply(lambda x: len(x.split(" ")))
test_df['num_description_words'] = test_df['description'].apply(lambda x: len(x.split(" ")))


# Даты
train_df['created'] = pd.to_datetime(train_df['created'])
test_df['created'] = pd.to_datetime(test_df['created'])

# Извлекаем время
train_df['created_year'] = train_df['created'].dt.year
test_df['created_year'] = test_df['created'].dt.year
train_df['created_month'] = train_df['created'].dt.month
test_df['created_month'] = test_df['created'].dt.month
train_df['created_day'] = train_df['created'].dt.day
test_df['created_day'] = test_df['created'].dt.day
train_df['created_hour'] = train_df['created'].dt.hour
test_df['created_hour'] = test_df['created'].dt.hour



# Фичи по ванной и спальням
# Разл в комнатах
train_df['room_difference'] = train_df['bedrooms'] - train_df['bathrooms']
test_df['room_difference'] = test_df['bedrooms'] - test_df['bathrooms']
# Общее количество
train_df['total_rooms'] = train_df['bedrooms'] + train_df['bathrooms']
test_df['total_rooms'] = test_df['bedrooms'] + test_df['bathrooms']
# Цена 1 добавляется чтобы исключить деление на ноль
train_df['price_per_room'] = train_df['price'] / (train_df['total_rooms'] + 1)
test_df['price_per_room'] = test_df['price'] / (test_df['total_rooms'] + 1)
# Цена за спальню
train_df['price_per_bedroom'] =train_df['price']/(train_df['bedrooms'] + 1)
test_df['price_per_bedroom'] =test_df['price']/(test_df['bedrooms'] +1)

# Цена за ванную
train_df['price_per_bathroom'] =train_df['price']/(train_df['bathrooms'] +1)
test_df['price_per_bathroom'] =test_df['price']/(test_df['bathrooms'] +1)

# Близость к метро
subway = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv') # сторонние данные
subway = subway[['Station Name', 'Station Latitude', 'Station Longitude']]
subway = subway.groupby(['Station Name']).mean().reset_index(drop=True)


# Функция минимального расстояния от метро
def get_subway_distance(location):
    distances = distance_pairs(location[0], location[1], subway['Station Latitude'], subway['Station Longitude'])    
    return min(distances)

# Расчет минимальной дистанции до метро
train_df['location'] = train_df[['latitude', 'longitude']].values.tolist()
test_df['location'] = test_df[['latitude', 'longitude']].values.tolist()

train_df['subway_distance'] = train_df['location'].apply(get_subway_distance)
test_df['subway_distance'] = test_df['location'].apply(get_subway_distance)


# Плотность по координатам https://www.kaggle.com/code/maheshak04/rh004
train_df["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test_df["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

vals = train_df['pos'].value_counts()
dvals = vals.to_dict()
train_df["density"] = train_df['pos'].apply(lambda x: dvals.get(x, vals.min()))
test_df["density"] = test_df['pos'].apply(lambda x: dvals.get(x, vals.min()))



# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["listing_id1"] = train_df["listing_id"] - 68119576.0
test_df["listing_id1"] =  test_df["listing_id"] - 68119576.0

train_df["total_days"] =   (train_df["created_month"] -4.0)*30 + train_df["created_day"] +  train_df["created_hour"] /25.0
test_df["total_days"] =(test_df["created_month"] -4.0)*30 + test_df["created_day"] +  test_df["created_hour"] /25.0        
train_df["diff_rank"]= train_df["total_days"]/train_df["listing_id1"]
test_df["diff_rank"]= test_df["total_days"]/test_df["listing_id1"]

# Фича из фото нужно скачать торрент файл
df_time = pd.read_csv('listing_image_time.csv')
train_df = train_df.merge(df_time, how='left', on='listing_id', suffixes=('', '_remove'))
test_df = test_df.merge(df_time, how='left', on='listing_id', suffixes=('', '_remove'))

In [220]:
# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["price0"] = (train_df["price"]%10).astype(int)
test_df["price0"] = (test_df["price"]%10).astype(int)

In [221]:
# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["price_latitude"] = (train_df["price"])/ (train_df["latitude"]+1.0)
test_df["price_latitude"] =  (test_df["price"])/ (test_df["latitude"]+1.0)

train_df["price_longtitude"] = (train_df["price"])/ (train_df["longitude"]-1.0)
test_df["price_longtitude"] =  (test_df["price"])/ (test_df["longitude"]-1.0)  

In [222]:
list_feat = ['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'latitude','listing_id', 'longitude',
       'manager_id', 'photos', 'price', 'street_address', 'interest_level',
       'distance_manh', 'distance_central_park', 'bronx', 'queens', 'brooklin_cometery', 'brooklin', 'clusters',
       'num_features', 'num_description_words', 'created_year',
       'created_month', 'created_day', 'created_hour', 'room_difference',
       'total_rooms', 'price_per_room', 'subway_distance', 
        "density", "price_latitude", "price_longtitude", 'total_days', 'diff_rank', 'time_stamp', 'price0', 
             'price_per_bedroom', 'price_per_bathroom']



list_test = ['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'latitude', 'listing_id', 'longitude',
       'manager_id', 'photos', 'price', 'street_address',
       'distance_manh', 'distance_central_park', 'bronx', 'queens', 'brooklin_cometery', 'brooklin', 'clusters',
       'num_features', 'num_description_words', 'created_year',
       'created_month', 'created_day', 'created_hour', 'room_difference',
       'total_rooms', 'price_per_room', 'subway_distance', 
        "density",
        "price_latitude", "price_longtitude", 'total_days', 'diff_rank', 'time_stamp', 'price0',
            'price_per_bedroom', 'price_per_bathroom']

train_df = train_df[list_feat]
test_df = test_df[list_test]

In [223]:
# Текстовые данные
#Колонки для чистки текста
cols_text = ['description', 'display_address', 'street_address', 'features']
#Преобразуем в тип str
train_df[cols_text] = train_df[cols_text].astype(str)

test_df[cols_text] = test_df[cols_text].astype(str)

#To lower case

train_df[cols_text] = train_df[cols_text].apply(lambda x: x.str.lower())
test_df[cols_text] = test_df[cols_text].apply(lambda x: x.str.lower())

#Remove punctuation
with_whitespace = ['&', '(', ')', "-", "_", ':', '=', '"', ',']
with_empty = ['.', "'", '`', '!', '*', '#', '/', '<', '>', 'br',
              ';', '$', '%', '|', '+', '?']


def replace_symbol(df, to_replace, replace_by):
    for symbol in to_replace:
        df = df.apply(lambda x: x.str.replace(symbol, replace_by, regex = True)) 
    return df

train_df[cols_text] = replace_symbol(train_df[cols_text], with_whitespace, ' ')
train_df[cols_text] = replace_symbol(train_df[cols_text], with_empty, '')

test_df[cols_text] = replace_symbol(test_df[cols_text], with_whitespace, ' ')
test_df[cols_text] = replace_symbol(test_df[cols_text], with_empty, '')

In [224]:
# Ищу элементы в текстовых данных
desc_feat = ['description', 'features']
#ссылки
train_df[desc_feat] = train_df[desc_feat].replace('\swww.\S*', ' weblink ', regex = True)
#почты
train_df[desc_feat] = train_df[desc_feat].replace('\s\S*@\S*', ' emailaddress', regex = True)
#время
train_df[desc_feat] = train_df[desc_feat].replace('\s\d{1,2}\s\d\d[ap]m', ' ampmtime', regex = True)
#номера
train_df[desc_feat] = train_df[desc_feat].replace('\s\d{2,4}\s\d{2,4}\s\d{2,4}', ' phonenumber', regex = True)


test_df[desc_feat] = test_df[desc_feat].replace('\swww.\S*', ' weblink ', regex = True)
#почты
test_df[desc_feat] = test_df[desc_feat].replace('\s\S*@\S*', ' emailaddress', regex = True)
#timeвремя
test_df[desc_feat] = test_df[desc_feat].replace('\s\d{1,2}\s\d\d[ap]m', ' ampmtime', regex = True)
#номера
test_df[desc_feat] = test_df[desc_feat].replace('\s\d{2,4}\s\d{2,4}\s\d{2,4}', ' phonenumber', regex = True)

In [225]:
# Чистка текста
adr_feat = ['display_address', 'street_address']


train_df[adr_feat] = train_df[adr_feat].replace(['\sst\s', '\sst$'], ' street', regex = True)
train_df[adr_feat] = train_df[adr_feat].replace(['\save\s', '\save$'], ' avenue', regex = True)

test_df[adr_feat] = test_df[adr_feat].replace(['\sst\s', '\sst$'], ' street', regex = True)
test_df[adr_feat] = test_df[adr_feat].replace(['\save\s', '\save$'], ' avenue', regex = True)


train_df[adr_feat] = train_df[adr_feat].replace(['\se\s', '^e\s'], ' east ', regex = True)
train_df[adr_feat] = train_df[adr_feat].replace(['\sw\s', '^w\s'], ' west ', regex = True)

test_df[adr_feat] = test_df[adr_feat].replace(['\se\s', '^e\s'], ' east ', regex = True)
test_df[adr_feat] =test_df[adr_feat].replace(['\sw\s', '^w\s'], ' west ', regex = True)

In [226]:
# Кодирую признаки
num_target = {'high':0, 'medium':1, 'low':2}
train_df['interest_level'] = train_df['interest_level'].apply(lambda x: num_target[x])

In [227]:
train_df = train_df.reset_index(drop=True)

In [228]:
X = train_df.drop('interest_level', axis=1)
y = train_df['interest_level']

In [229]:
from catboost import CatBoostClassifier, Pool, metrics, cv


In [230]:

from sklearn.model_selection import StratifiedKFold
SEED = 42
FOLDS = 7

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

In [231]:
# Кроссвалидация


cv_test_preds = np.zeros((len(test_df), 3))

cat_features = ['created_day', 'created_month','manager_id', 'building_id', 'display_address']
text_features = ['description','street_address' , 'features']
for train_idx, val_idx in skf.split(X, y):
    X_train, y_train = X.iloc[train_idx, :], y[train_idx]
    X_val, y_val = X.iloc[val_idx, :], y[val_idx]
    
    train = Pool(data=X_train, 
             label=y_train,            
             feature_names=list(X_train.columns),
             cat_features=cat_features,
             text_features = text_features)

    val = Pool(data=X_val, 
               label=y_val,
               feature_names=list(X_val.columns),
               cat_features=cat_features,
               text_features = text_features)

    model = CatBoostClassifier(random_seed = 17,
                              iterations = 1000,
                            thread_count = -1, 
                            verbose = 100,
                              bootstrap_type="Bernoulli",
                            loss_function='MultiClass',
                          task_type="GPU")
    
    model.fit(train,
            early_stopping_rounds=50,
             eval_set=val,
             use_best_model=True)

    Test = Pool(data=test_df,
            cat_features=cat_features,
            text_features = text_features)
    
    preds = model.predict_proba(Test)
    
    
    cv_test_preds += model.predict_proba(Test) / FOLDS
    
    


Learning rate set to 0.153494
0:	learn: 0.9798318	test: 0.9809291	best: 0.9809291 (0)	total: 223ms	remaining: 3m 42s
100:	learn: 0.5156126	test: 0.5304539	best: 0.5304539 (100)	total: 5.03s	remaining: 44.8s
200:	learn: 0.4901054	test: 0.5232505	best: 0.5232505 (200)	total: 9.05s	remaining: 36s
300:	learn: 0.4719124	test: 0.5194544	best: 0.5194544 (300)	total: 13.7s	remaining: 31.9s
400:	learn: 0.4575862	test: 0.5172345	best: 0.5172345 (400)	total: 17.5s	remaining: 26.1s
500:	learn: 0.4428320	test: 0.5160879	best: 0.5160811 (498)	total: 21.4s	remaining: 21.3s
600:	learn: 0.4311557	test: 0.5159010	best: 0.5156582 (592)	total: 25.2s	remaining: 16.7s
700:	learn: 0.4196606	test: 0.5157836	best: 0.5154752 (654)	total: 29.1s	remaining: 12.4s
bestTest = 0.5154752124
bestIteration = 654
Shrink model to first 655 iterations.
Learning rate set to 0.153494
0:	learn: 0.9800798	test: 0.9791356	best: 0.9791356 (0)	total: 85.5ms	remaining: 1m 25s
100:	learn: 0.5179548	test: 0.5292147	best: 0.5292147 (

In [232]:
predictions = pd.DataFrame(cv_test_preds)

sub = pd.DataFrame()
sub["listing_id"] = test_df.listing_id.values
sub['high'] = predictions[0]
sub['medium'] = predictions[1]
sub['low'] = predictions[2]

sub.to_csv('guzeev_cross_newpr_7_fold_new_room.csv', index = False)

### Важность фич

In [233]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,description,15.670987
1,manager_id,9.178532
2,features,8.972821
3,price_per_room,7.732289
4,building_id,6.692271
5,price_longtitude,4.995111
6,time_stamp,4.764185
7,price_per_bedroom,4.432489
8,distance_manh,3.710011
9,price,3.464551


##### 1.Не смотря на то, что даные более менее чистые (не имеют пропусков). Данные содержат ошибки:
 * Нулевое количество ванных комнат
 * Экстремальное количество ванных комнат 112 и 20 (при заполнении полагаю имелось в виду 1 и 2)
 * География по координатам содержала ошибочные данные. Также были совсем другие регионы
 * Цены тоже имели ошибочные данные или почти нулевая цена или заоблачная
 
##### 2. Идеи по Фич Инжинирингу:
 * Создание фич из отличия количества спален и ванных комнат
 * Цена за комнату
 * Цена за спальню и.т.д
 * Можно сделать регрессор зависмости ванных комнат и спален и найти отношение к реальной цене
 * Была идея разделить признаки столбец `features` по темам, разбивка на семь тем результатов не принесла, тогда написал функцию по оптимальному количеству тем, но функция работала долго (не дождался).
 * Фичи по дистанции от центральных мест дало прирост. За центральные места были взяты: финансовый квартал Манхэтэна и центральный парк
 * Одной из фич была кластеризация по координатам. Предполагал, что число кластеров должно быть равно числу станций метро=476, но лучше результат показал небольшое число кластеров 2-3. Тут представлена кластеризация Kmeans, но таже делал HDBSCAN
 * Как числовые данные были взяты количество признаков или слов, также и с фото
 
#### Модель
 * Настройку гиперпараметров не делал. Метрики классификации не показывал. Интересовал score в kaggle
 * Чистка TRAIN и TEST не принесла результатов, лучше модель работает на сырых данных
 * Или по экстремальным значениям данных извлекать больше сторонней информации. Например, пропущенные координаты можно было восстановить по адресу. Одной из фич можно было сделать расстояние от метро.
 * Конечный SCORE получился 0.54791
 
 * ***********
 * Catboost показал примерно такие же результаты
 * Фича привязка к метро, и дистанцию к самому бедному району (БРОНКС) дала прирост к SCORE около 0.5%
 * Конечный SCORE составил `0.54515` был `0.54834`
 * ***********
 #### ОБНОВЛЕНО 12.12.2022
 
 1. Были взяты некоторые фичи с этого кернела (указаны в комментариях)
 2. Хорошо увеличил скор при использовании времени создания картинок (торрент файл)
 3. Score увеличился при использовании catboost
 4. Использование коэффициента вариации (что-то похожее) при кодировании менеджеров для модели не принесли результатов
 5. Чистка координат по граничным зонам принесла результаты
 6. Границу для чистки по price необходимо "подлавливать"
 7. Привязка к другим районам дала прибавку к score. Где-то привязывал к кладбищу.
 
 ###### Возможные дальнейшие шаги:
 1. Сделать новые фичи на основе manager_id, building_id, итд. То есть копать больше в сторону менеджеров, building_id итд
 2. Делал по большей части в Catboost, так как Catboost сам кодирует текстовые и категориальные данные.
 3. Хотел затюнить модель `grid_search`, но даже с хорошей видеокартой считает долго.Не дождался. 
 